In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
from datetime import datetime
pymysql.install_as_MySQLdb()

In [2]:
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [5]:
course_upload = pd.read_sql("""
                            SELECT * 
                            FROM course""", conn)
course_upload.head()

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [6]:
for index, row in course_upload.iterrows():

    record = {
        'ID_Course__c': row['ID_Course'],
        'CourseCode__c': row['CourseCode'],
        'CourseName__c': row['CourseName'],
        'CreditHours__c': row['CreditHours'],
        'BootCampCourse__c': row['BootCampCourse']}
    
    try:
        sf.course__c.create(record)
    except Exception as e:
        print(e)

In [7]:
class_start = pd.read_sql( """
                            SELECT 
                            class.ID_Class,
                            class.ID_Course,
                            class.Section,
                            class.StartDate,
                            class.EndDate,
                            course.CourseCode
                            From class
                            Join course
                            On class.ID_Course = course.ID_Course""", conn)

class_start['StartDate'] = pd.to_datetime(class_start['StartDate']).dt.date
class_start['EndDate'] = pd.to_datetime(class_start['EndDate']).dt.date
class_start = class_start.rename(columns={"CourseCode": "CourseCode__c"})
class_start.head()

,ID_Class,ID_Course,Section,StartDate,EndDate,CourseCode__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,BC-DATAVIZ
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,BC-DATAVIZ
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,BC-WEBDEV
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,BC-UIUX


In [8]:
course_list = []
course_lookup = sf.query_all_iter("SELECT Name, CourseCode__C from course__c")

for row in course_lookup:
    record = {'course__c': row['Name'],
              'CourseCode__c': row['CourseCode__c']}
    
    course_list.append(record)

course_list_df = pd.DataFrame(course_list)
course_list_df

,course__c,CourseCode__c
0,a063h0000025G5c,CIS-430
1,a063h0000025G5I,BC-UIUX
2,a063h0000025G5S,CIS-405
3,a063h0000025G58,BC-DATAVIZ
4,a063h0000025G5X,CIS-438
5,a063h0000025G5D,BC-WEBDEV
6,a063h0000025G5h,CIS-435
7,a063h0000025G5N,CIS-349


In [9]:
class_upload = pd.merge(class_start, course_list_df, how = 'left')
class_upload

,ID_Class,ID_Course,Section,StartDate,EndDate,CourseCode__c,course__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14,BC-DATAVIZ,a063h0000025G58
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08,BC-DATAVIZ,a063h0000025G58
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07,BC-WEBDEV,a063h0000025G5D
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07,BC-UIUX,a063h0000025G5I


In [10]:
for index, row in class_upload.iterrows():

    record = {
        'ID_Class__c': row['ID_Class'],
        'ID_Course__c': row['ID_Course'],
        'Section__c': row['Section'],
        'StartDate__c': str(row['StartDate']),
        'EndDate__c': str(row['EndDate']),
        'course__c': row['course__c']}
    
    try:
        sf.class__c.create(record)
    except Exception as e:
        print(e)

In [15]:
student_upload = pd.read_sql("""
                            SELECT * 
                            FROM student""", conn)

student_upload['BirthDate'] = pd.to_datetime(student_upload['BirthDate']).dt.date

student_upload.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,NaT,F
1,34,25003514,Multak,Ilana,Cecille,NaT,F
2,35,25005833,Murillo,Jessica,Dorothy,NaT,F
3,36,25002589,Romanowski,Kandra,Genevieve,NaT,F
4,37,25007185,Hoffer,Katherine,Lynnette,NaT,F


In [19]:
for index, row in student_upload.iterrows():

    record = {
        'ID_Student__c': row['ID_Student'],
        'StudentID__c': row['StudentID'],
        'LastName__c': row['LastName'],
        'FirstName__c': row['FirstName'],
        'MiddleName__c': row['MiddleName'],
        'Gender__c': row['Gender']}
    
    try:
        sf.student__c.create(record)
    except Exception as e:
        print(e)

In [29]:
classparticipant_start = pd.read_sql( """
                                    SELECT 
                                    classparticipant.ID_ClassParticipant,
                                    classparticipant.ID_Student,
                                    classparticipant.ID_Class,
                                    classparticipant.StartDate,
                                    classparticipant.EndDate,
                                    class.Section
                                    From classparticipant
                                    Join class
                                    On classparticipant.ID_Class = class.ID_Class """, conn)

classparticipant_start['StartDate'] = pd.to_datetime(classparticipant_start['StartDate']).dt.date
classparticipant_start['EndDate'] = pd.to_datetime(classparticipant_start['EndDate']).dt.date
#classparticipant = classparticipant.rename(columns={"CourseCode": "CourseCode__c"})
classparticipant_start.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,Section
0,1,33,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0
1,2,34,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0
2,3,35,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0
3,4,62,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0
4,5,36,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0


In [32]:
student_list = []
student_lookup = sf.query_all_iter("SELECT Name, ID_Student__C from student__c")

for row in student_lookup:
    record = {'student__c': row['Name'],
              'ID_Student': row['ID_Student__c']}
    
    student_list.append(record)

student_list_df = pd.DataFrame(student_list)
student_list_df.head()

,student__c,ID_Student
0,a0D3h000000OQbj,52.0
1,a0D3h000000OQbt,54.0
2,a0D3h000000OQbK,46.0
3,a0D3h000000OQbZ,49.0
4,a0D3h000000OQba,60.0


In [36]:
class_list = []
class_lookup = sf.query_all_iter("SELECT Name, Section__c from class__c")

for row in class_lookup:
    record = {'class__c': row['Name'],
              'Section': row['Section__c']}
    
    class_list.append(record)

class_list_df = pd.DataFrame(class_list)
class_list_df.head()

,class__c,Section
0,a083h000000nlx9,GWARL201905UIUX3
1,a083h000000nlx4,GWARL201905WEB3
2,a083h000000nlwz,GWDC201805DATA3
3,a083h000000nlwu,GWU-ARL-DATA-PT-09-0


In [37]:
classparticipant_upload = pd.merge(classparticipant_start, student_list_df, how = 'left')
classparticipant_upload.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,Section,student__c
0,1,33,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaH
1,2,34,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaM
2,3,35,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaR
3,4,62,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQcN
4,5,36,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaW


In [39]:
classparticipant_upload = pd.merge(classparticipant_upload, class_list_df, how = 'left')
classparticipant_upload.head()

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate,Section,student__c,class__c
0,1,33,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaH,a083h000000nlwu
1,2,34,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaM,a083h000000nlwu
2,3,35,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaR,a083h000000nlwu
3,4,62,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQcN,a083h000000nlwu
4,5,36,1,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,a0D3h000000OQaW,a083h000000nlwu


In [45]:
for index, row in classparticipant_upload.iterrows():

    record = {
        'ID_ClassParticipant__c': row['ID_ClassParticipant'],
        'ID_Student__c': row['ID_Student'],
        'ID_Class__c': row['ID_Class'],
        'StartDate__c': str(row['StartDate']),
        'student__c': row['student__c'],
        'class__c': row['class__c']}
    try:
        sf.classparticipant__c.create(record)
    except Exception as e:
        print(e)

In [47]:
staff_upload = pd.read_sql("""
                            SELECT * 
                            FROM staff""", conn)

staff_upload['BirthDate'] = pd.to_datetime(staff_upload['BirthDate']).dt.date
staff_upload.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [50]:
for index, row in staff_upload.iterrows():

    record = {
        'ID__c': row['ID_Staff'],
        'EmployeeID__c': row['EmployeeID'],
        'LastName__c': row['LastName'],
        'FirstName__c': row['FirstName'],
        'MiddleName__c': row['MiddleName'],
        'BirthDate__c': str(row['BirthDate'])}
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

In [51]:
class_list2 = []
class_lookup2 = sf.query_all_iter("SELECT Name, ID_Class__c from class__c")

for row in class_lookup2:
    record = {'class__c': row['Name'],
              'ID_Class': row['ID_Class__c']}
    
    class_list2.append(record)

class_list2_df = pd.DataFrame(class_list2)
class_list2_df.head()

,class__c,ID_Class
0,a083h000000nlx9,4.0
1,a083h000000nlx4,3.0
2,a083h000000nlwz,2.0
3,a083h000000nlwu,1.0


In [53]:
staff_list = []
staff_lookup = sf.query_all_iter("SELECT Name, ID__c from staff__c")

for row in staff_lookup:
    record = {'staff__c': row['Name'],
              'ID_Staff': row['ID__c']}
    
    staff_list.append(record)

staff_list_df = pd.DataFrame(staff_list)
staff_list_df.head()

,staff__c,ID_Staff
0,a0B3h000001Fy7M,4.0
1,a0B3h000001Fy7C,2.0
2,a0B3h000001Fy77,1.0
3,a0B3h000001Fy7H,3.0


In [55]:
staffassignment_start = pd.read_sql("""
                            SELECT * 
                            FROM staffassignment""", conn)
staffassignment_start.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [61]:
staffassignment_upload = pd.merge(staffassignment_start, class_list2_df, how = 'left')
staffassignment_upload.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,class__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a083h000000nlwu
1,2,4,1,Teacher Assistant,2020-03-16,None,a083h000000nlwu
2,3,3,1,Instructor,2020-03-16,None,a083h000000nlwu


In [62]:
staffassigment_upload = pd.merge(staffassignment_upload, staff_list_df, how = 'left')
staffassigment_upload.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,class__c,staff__c
0,1,1,1,Teacher Assistant,2020-03-16,None,a083h000000nlwu,a0B3h000001Fy77
1,2,4,1,Teacher Assistant,2020-03-16,None,a083h000000nlwu,a0B3h000001Fy7M
2,3,3,1,Instructor,2020-03-16,None,a083h000000nlwu,a0B3h000001Fy7H


In [64]:
for index, row in staffassigment_upload.iterrows():

    record = {
        'ID_Staff__c': row['ID_Staff'],
        'ID_Class__c': row['ID_Class'],
        'Role__c': row['Role'],
        'class__c': row['class__c'],
        'staff__c': row['staff__c'],
        'StartDate__c': str(row['StartDate'])}
    
    try:
        sf.staffassignment__c.create(record)
    except Exception as e:
        print(e)